# Business issue

H&M Group is a family of brands and businesses with 53 online markets and approximately 4,850 stores. Our online store offers shoppers an extensive selection of products to browse through. But with too many choices, customers might not quickly find what interests them or what they are looking for, and ultimately, they might not make a purchase. To enhance the shopping experience, product recommendations are key. More importantly, helping customers make the right choices also has a positive implications for sustainability, as it reduces returns, and thereby minimizes emissions from transportation.

In this competition, H&M Group invites you to develop product recommendations based on data from previous transactions, as well as from customer and product meta data. The available meta data spans from simple data, such as garment type and customer age, to text data from product descriptions, to image data from garment images.

There are no preconceptions on what information that may be useful – that is for you to find out. If you want to investigate a categorical data type algorithm, or dive into NLP and image processing deep learning, that is up to you.

# Business request

For this challenge you are given the purchase history of customers across time, along with supporting metadata. Your challenge is to predict what articles each customer will purchase in the 7-day period immediately after the training data ends. Customer who did not make any purchase during that time are excluded from the scoring.

# Data

images/ - a folder of images corresponding to each article_id; images are placed in subfolders starting with the first three digits of the article_id; note, not all article_id values have a corresponding image.

articles.csv - detailed metadata for each article_id available for purchase

customers.csv - metadata for each customer_id in dataset

transactions_train.csv - the training data, consisting of the purchases each customer for each date, as well as additional information. Duplicate rows correspond to multiple purchases of the same item. Your task is to predict the article_ids each customer will purchase during the 7-day period immediately after the training data period.


# Mount G. drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3.10 install virtualenv
!virtualenv env
!source /content/env/bin/activate

created virtual environment CPython3.10.12.final.0-64 in 419ms
  creator CPython3Posix(dest=/content/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==74.1.2, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


# Importing modules

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
import missingno as msno

In [5]:
import sklearn
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [6]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import TensorDataset, random_split
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.io import read_image

In [7]:
import torch.nn.functional as F
from  torch.nn.functional import one_hot
import torch.optim as optim
from torch.optim import Adam

In [8]:
from xgboost import XGBClassifier

# Loading datasets

## Paths and function

In [9]:
nrows = 8000000

In [10]:
class Load():
    def __init__(self, path, nrows=nrows):
        self.path = path
        self.nrows = nrows
    def read(self):
        df = pd.read_csv(self.path, sep=',', nrows=self.nrows)
        return df

In [11]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [12]:
#path_cust = '/Users/olivierdebeyssac/Python_demand_forecasting/h_m_data/customers.csv'
#path_art = '/Users/olivierdebeyssac/Python_demand_forecasting/h_m_data/articles.csv'
#path_transac = '/Users/olivierdebeyssac/Python_demand_forecasting/h_m_data/transactions_train.csv'

In [13]:
path_cust = '/content/drive/MyDrive/Colab Notebooks/Python_demand_forecasting/h_m_data/customers.csv'
path_art = '/content/drive/MyDrive/Colab Notebooks/Python_demand_forecasting/h_m_data/articles.csv'
path_transac = '/content/drive/MyDrive/Colab Notebooks/Python_demand_forecasting/h_m_data/transactions_train.csv'

## Customer data

In [14]:
data_cust = Load(path_cust)
df_cust = data_cust.read()
df_cust.info()

KeyboardInterrupt: 

In [ ]:
df_cust.shape

In [ ]:
df_cust.isnull().mean()

In [ ]:
df_cust.head()

- We see high rate missing values for features:
  - FN
  - Active
- We will treat missing values later in the notebook

## Articles data

In [ ]:
data_art = Load(path_art)
df_art = data_art.read()
df_art.info()

In [ ]:
df_art.isnull().mean()

In [ ]:
df_art.head()

- df_art has no missing values.

## Transaction data

In [ ]:
data_transac = Load(path_transac)
df_transac = data_transac.read()
df_transac.info()

In [ ]:
df_transac.isnull().mean()

In [ ]:
df_transac.head()

- df_transac has no missing values.

# Data visualisation.

In [ ]:
age_min = df_cust['age'].min()
age_med = df_cust['age'].median()
age_max = df_cust['age'].max()
print('age min: {}, age_med: {}, age max: {}'.format(age_min, age_med, age_max))

In [ ]:
"""
Histogram of age
"""

fig = px.histogram(df_cust['age'])
fig.show()

In [ ]:
"""
data distribution of age
"""

fig = px.box(df_cust, x="age")
fig.show()

In [ ]:
price_min = df_transac['price'].min()
price_med = df_transac['price'].median()
price_max = df_transac['price'].max()
print('price min: {}, price_med: {}, price max: {}'.format(price_min, price_med, price_max))

In [ ]:
"""histogram of price

No display with full dataset...so taking first 10000 rows"""

fig = px.histogram(df_transac[0:100000], x='price')
fig.show()

In [ ]:
"""data distribution of price"""

fig = px.box(df_transac[0:100000], x="price")
fig.show()

# Join

- At this point we need to join multiple datasets and get a single one.
- This will help to keep consistency along next data processing steps.

- We first join df_transac and df_cust.

In [ ]:
df_1= pd.merge(df_transac, df_cust, on='customer_id', how='left')

In [ ]:
df_1.info()

- Then we join df_1 and df_art to gether.

In [ ]:
df_glob = pd.merge(df_1, df_art, on='article_id', how='left')

In [ ]:
df_glob.info()

In [ ]:
df_glob.head()

In [ ]:
df_glob.isnull().mean()

# Clustering with RFM technique.


- RFM should help in reducing number of samples mainly by focusing on relevant ones.

## Recency.

In [ ]:
df_glob['t_dat'] = pd.to_datetime(df_glob['t_dat'])
most_recent_date = df_glob['t_dat'].max()

In [ ]:
df_glob['recency'] = most_recent_date - df_glob['t_dat']
df_glob['recency'] = df_glob['recency'].dt.days

In [ ]:
df_glob['recency'][0:2]

## Frequency.

In [ ]:
frequency_data = df_glob.groupby('customer_id')['article_id'].count().reset_index()
frequency_data.rename(columns={'article_id': 'frequency'}, inplace=True)

In [ ]:
df_glob = pd.merge(df_glob, frequency_data, on='customer_id', how='left')

## Monetory.

In [ ]:
monetary_data = df_glob.groupby('customer_id')['price'].sum().reset_index()
monetary_data.rename(columns={'price': 'monetary'}, inplace=True)

In [ ]:
df_glob = pd.merge(df_glob, monetary_data, on='customer_id', how='left')

## RFM Scores.

In [ ]:
recency_score = pd.qcut(df_glob['recency'], q=5, labels=[5, 4, 3, 2, 1])
frequency_score = pd.qcut(df_glob['frequency'], q=5, labels=[1, 2, 3, 4, 5])
monetary_score = pd.qcut(df_glob['monetary'], q=5, labels=[1, 2, 3, 4, 5])

In [ ]:
df_glob['recency_score'] = recency_score
df_glob['frequency_score'] = frequency_score
df_glob['monetary_score'] = monetary_score

In [ ]:
df_glob.iloc[0:5, -5:]

## Segmentation.

In [ ]:
df_glob['rfm_score'] = df_glob['recency_score'].astype(int) + df_glob['frequency_score'].astype(int) + df_glob['monetary_score'].astype(int)
cols = ['customer_id', 'recency', 'frequency', 'monetary', 'rfm_score']
agg_fn = lambda x: x.sum(axis=0)

data_rfm = df_glob.loc[:, cols].groupby(['customer_id']).agg(agg_fn)
data_rfm.head()

In [ ]:
segment_labels = ['low_values', 'mid_values', 'high_values']
data_rfm['rfm_segment'] = pd.qcut(data_rfm['rfm_score'], q=3, labels=segment_labels)
data_rfm.head()

## Display RFM.

In [ ]:
"""how much of customers per cluster ? """
s_cust_per_cluster = data_rfm['rfm_segment'].value_counts()
s_check = s_cust_per_cluster.sum()

print(f'low_values: {s_cust_per_cluster["low_values"]}, percent of total: {s_cust_per_cluster["low_values"]/s_check}')
print(f'mid_values: {s_cust_per_cluster["mid_values"]}, percent of total: {s_cust_per_cluster["mid_values"]/s_check}')
print(f'high_values: {s_cust_per_cluster["high_values"]}, percent of total: {s_cust_per_cluster["high_values"]/s_check}')
print('---')
s_check = s_cust_per_cluster.sum()
print(f'check: {s_check}')

In [ ]:
fig = px.bar(s_cust_per_cluster, x=s_cust_per_cluster.index,
             y=s_cust_per_cluster.values,
             color=s_cust_per_cluster.index,
             title='Unique customers per cluster')
fig.show()

- As a conclusion of RFM analysis and as per the objective of taking out of data the ones not relevant enough to the compagny, we could decide to keep mid-values and high-values segments only.
- Let's see what would be the results working with k-means methodology.

# Clustering with K-means.

In [ ]:
scaler = StandardScaler()

In [ ]:
data_rfm_sc = scaler.fit_transform(data_rfm[['recency', 'frequency', 'monetary', 'rfm_score']].values)

In [ ]:
data_rfm_sc[0:5, :]

## Importing librairies.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
kmeans = KMeans()

In [ ]:
elbow = KElbowVisualizer(kmeans,k=(2, 20))
elbow.fit(data_rfm_sc)
elbow.show()

In [ ]:
n = elbow.elbow_value_
kmeans = KMeans(n_clusters=n).fit(data_rfm_sc)
centers = kmeans.cluster_centers_
clusters = kmeans.labels_

In [ ]:
data_rfm_sc = np.concatenate([data_rfm_sc, clusters.reshape(-1, 1)], axis=1)
print(f'{data_rfm_sc[0:5, :]}')
print(f'shape: {data_rfm_sc.shape}')

In [ ]:
data_rfm_sc = np.concatenate([np.asarray(list(data_rfm.index)).reshape(-1, 1), data_rfm_sc], axis=1)
print(f'{data_rfm_sc[0:5, :]}')
print(f'shape: {data_rfm_sc.shape}')

In [ ]:
df_rfm_sc = pd.DataFrame(data=data_rfm_sc, columns=['customer_id', 'recency', 'frequency', 'monetary', 'rfm_score', 'clusters'])
df_rfm_sc.head()

In [ ]:
df_rfm_sc[['recency', 'frequency', 'monetary', 'rfm_score', 'clusters']] = df_rfm_sc[['recency', 'frequency', 'monetary', 'rfm_score', 'clusters']].astype(float)
df_rfm_sc[['clusters']] = df_rfm_sc[['clusters']].astype(int)

In [ ]:
df_rfm_sc.info()

In [ ]:
df_rfm_sc['clusters'] = df_rfm_sc['clusters'] + 1


In [ ]:
df_rfm_sc[df_rfm_sc['clusters'] == 1][0:5]

In [ ]:
df_rfm_sc[df_rfm_sc['clusters'] == 6][0:5]

In [ ]:
sns.scatterplot(data=df_rfm_sc, x='clusters', y='rfm_score', hue='clusters', palette='deep')
plt.tight_layout()
plt.xlabel('cluster')
plt.ylabel('rfm_score')
plt.title('RFM score & clusters')
plt.show()

In [ ]:
"""Counting number of customers per cluster"""

def count_cust_per_cluster(df):
  d = {}
  for cluster in df['clusters'].unique():
    nb_occurencies = len(df[df['clusters'] == cluster])
    percent_occurencies = nb_occurencies / len(df)
    print(f'--> cluster: {cluster}, nb_occurencies: {nb_occurencies}, percent_occurencies: {percent_occurencies}')
    print('---')
    d[cluster] = percent_occurencies
  print(f'check: {sum(d.values())}')
  d = pd.DataFrame(data=d, columns=list(d.keys()), index=range(0,1))
  d = d.transpose()
  d.rename(columns={0: 'percent_occurencies'}, inplace=True)
  return d

In [ ]:
df_0 = count_cust_per_cluster(df_rfm_sc)

In [ ]:
df_0 = df_0.reset_index(drop=False)
df_0.rename(columns={'index': 'cluster'}, inplace=True)
df_0

In [ ]:
fig = px.bar(df_0, x='cluster', y='percent_occurencies',
             color='cluster',
             title='Percent of occurencies per cluster',
             hover_name='cluster',
             hover_data='percent_occurencies')
fig.show()

**- We take decision to take cluster#1 out of data**

In [ ]:
"""merge clusters with df_glob"""

final_rfm_data = df_rfm_sc[['customer_id', 'clusters']]
df_glob_clustered = pd.merge(df_glob, final_rfm_data, on='customer_id', how='left')

In [ ]:
df_glob_clustered.info()

In [ ]:
"""features to drop"""
print('---')
print(f'features before deletion: {len(df_glob_clustered.columns)}')
features_to_drop = ['recency', 'frequency', 'monetary']

print('---')
df_glob_clustered.drop(features_to_drop, axis=1, inplace=True)
print(f'features after deletion: {len(df_glob_clustered.columns)}')


In [ ]:
"""rows to drop:
correspond to clusters 1
"""
print(f'nb rows before deletion: {len(df_glob_clustered)}')
cluster_1 = df_glob_clustered[df_glob_clustered['clusters'] == 1]

rows_to_drop = cluster_1.index
df_glob_clustered.drop(rows_to_drop, axis=0, inplace=True)
print(f'nb rows after deletion: {len(df_glob_clustered)}')
print('---')
nb_rows_deleted = len(cluster_1)
print(f'% rows deleted: {nb_rows_deleted/len(df_glob_clustered)} %')


In [ ]:
"""reset index"""
df_glob_clustered.reset_index(drop=True, inplace=True)

In [ ]:
df_glob_clustered.info()

# Graphs.

- Because of computing limitations we cannot grab the whole dataset but just a small part of it.
- Unfortunately, we display some months instead of some years.



1.   Number of transactions par day.
2.   Number of transactions in function of age.
3.   Price of articles per age category.
4.   Cumulative sales in volume.
5.   Cumulative sales in volume per customer and per age category.



In [ ]:
"""are there customers with no purchases ?"""
df_glob_clustered[df_glob_clustered.groupby('customer_id')['price'].transform('count') == 0]

In [ ]:
"""
start & ending dates in dataset
"""
print(f'start date: {df_glob_clustered["t_dat"].min()}')
print(f'end date: {df_glob_clustered["t_dat"].max()}')


In [ ]:
"""
price min & max
"""
print(f'price min: {df_glob_clustered["price"].min()}, price max: {df_glob_clustered["price"].max()}')

In [ ]:
vol = df_glob_clustered.groupby('t_dat')['article_id'].count()
fig = go.Figure(data=go.Scatter(x=vol.index, y=vol.values),
                layout=go.Layout(title=go.layout.Title(text="Number of transactions per day")))
fig

In [ ]:
vol_age = df_glob_clustered.groupby('age')['article_id'].count()
fig = go.Figure(data=go.Scatter(x=vol_age.index, y=vol_age.values),
                layout=go.Layout(title=go.layout.Title(text="Number of transactions per age")))
fig

In [ ]:
"""build age categories and make scatter of pairs of age category and price"""
df_glob_clustered['age_cat'] = pd.cut(df_glob_clustered['age'], bins=list(np.arange(10, 95, 4)))
#scatter_age_cat = df_glob[['age_cat', 'price']]

sns.scatterplot(x=df_glob_clustered['age_cat'].astype(str), y=df_glob_clustered['price'], data=df_glob_clustered[['age_cat', 'price']], hue=list(df_glob_clustered['age_cat']))
plt.tight_layout()
plt.xlabel('age category')
plt.ylabel('price')
plt.title('Price per age category')
plt.xticks(rotation=45)
plt.show()


In [ ]:
"""cumulative sales"""
df_glob_clustered['cum_sales'] = df_glob_clustered['price'].cumsum()
fig = sns.lineplot(x='t_dat', y='cum_sales', data=df_glob_clustered)
plt.tight_layout()
plt.xlabel('date')
plt.ylabel('cumulative sales')
plt.show()

In [ ]:
df_glob_clustered['sales_by_customer'] = df_glob_clustered.groupby('customer_id')['price'].transform('sum')
df_glob_1 = df_glob_clustered.drop_duplicates(subset=['customer_id'])
fig = sns.barplot(x='t_dat', y='sales_by_customer', data=df_glob_1)
plt.tight_layout()
plt.xlabel('date')
plt.ylabel('sales by customer')
plt.xticks(rotation=45)
plt.show()

# Process missing values.

## Check missings.

- We need to focus on features that carry information. Meaning features with too much missing values are not informative enough and have to be taken out.

If missing rate above 50%, consider to delete features as we would not get data enough.

In [ ]:
def miss(df, thres):
  l_miss = []

  features = list(df.columns)
  lengh = len(df)
  for feature in features:
    missings = df.loc[:,feature].isnull().mean()

    if missings > thres:
      l_miss.append(feature)
  new_df = df.drop(l_miss, axis=1)
  print('info--nb cols before: {}, nb cols after: {}'.format(len(df.columns), len(new_df.columns)))
  print('suppressed features: {}'.format(l_miss))
  return l_miss, new_df


In [ ]:
supp_cols, new_df_glob = miss(df_glob_clustered,thres=0.5)

## Informative/Non informative features.

- We will take out features we estimate not to be informative enough.
- Here are full features list:

In [ ]:
df_cols = list(new_df_glob.columns)
print('features: {}'.format(df_cols))

- Taken features list we have to check feature category, missing values, informative capability.

In [ ]:
# ==> club_member_status
def know_more(feature_text):

  return print('dtype: {}, missing: {}, nb of unique: {}, unique: {}'.format(new_df_glob[feature_text].dtype,
                                                                    new_df_glob[feature_text].isnull().mean(),
                                                                    new_df_glob[feature_text].nunique(),
                                                                    new_df_glob[feature_text].unique()))

In [ ]:
know_more(feature_text='club_member_status')

- _<font color="red"> --> 'club_member_status' appear as not informative feature.</font>_
- _<font color="red"> --> We take out that feature.</font>_

In [ ]:
def supp_feature(df, feature):
  print('before suppress feature, nb cols: {}'.format(len(list(df.columns))))

  df_after = df.drop([feature], axis=1)

  print('after suppress feature, nb cols: {}'.format(len(list(df_after.columns))))
  return df_after

new_df_glob = supp_feature(new_df_glob, 'club_member_status')


In [ ]:
know_more('fashion_news_frequency')

- _<font color="red"> --> 'fashion_news_frequency' appear as not informative feature.</font>_
- _<font color="red"> --> We take out that feature.</font>_

In [ ]:
new_df_glob = supp_feature(new_df_glob, 'fashion_news_frequency')


In [ ]:
know_more('prod_name')

- _<font color="red"> --> 'prod_name' appear as informative feature.</font>_
- _<font color="red"> --> We keep that feature in.</font>_

In [ ]:
know_more('product_type_name')

- _<font color="red"> --> 'product_type_name' appear as informative feature.</font>_
- _<font color="red"> --> We keep that feature in.</font>_

In [ ]:
know_more('product_group_name')

- _<font color="red"> --> 'product_group_name' appear as informative feature.</font>_
- _<font color="red"> --> We keep that feature in.</font>_

In [ ]:
know_more('garment_group_name')

- _<font color="red"> --> 'garment_group_name' appear as informative feature.</font>_
- _<font color="red"> --> We keep that feature in.</font>_

In [ ]:
know_more('colour_group_name')

- _<font color="red"> --> 'colour_group_name' appear as informative feature.</font>_
- _<font color="red"> --> We keep that feature in.</font>_


In [ ]:
know_more('detail_desc')

- _<font color="red"> --> 'detail_desc' appear as informative feature.</font>_
- _<font color="red"> --> We keep that feature in.</font>_

---



- _<font color="red"> --> features with 'code', 'name' (with exceptions) or 'id' are considered as not relevant to characterize products.</font>_
- _<font color="red"> --> We take out those features.</font>_

## Deleting features with 'code', or 'name' or 'id'.

In [ ]:
#df_art.drop(['product_code'	,'graphical_appearance_no','product_type_no','colour_group_code','perceived_colour_value_id','perceived_colour_master_id','department_no','index_code','index_group_no','section_no','garment_group_no' ], axis=1, inplace=True)

In [ ]:
def supp_non_inf(df):
  s_1 = '_no'
  s_2 = '_code'

  cols_tobe_deleted = [element for element in list(new_df_glob.columns) if (s_1 in element) or (s_2 in element)]
  df_after = df.drop(cols_tobe_deleted, axis=1)
  print('before supp: {}, after supp: {}'.format(len(df.columns), len(df_after.columns)))
  print('remaining features: {}'.format(list(df_after.columns)))
  return df_after

In [ ]:
df_glob_after = supp_non_inf(new_df_glob)

In [ ]:
def supp_other_non_inf(df):
  cols = list(df.columns)
  print('nb cols before: {}'.format(len(cols)))
  # elements with 'name' in it we have to keep
  s_01 = 'prod_name'
  s_02 = 'product_type_name'
  s_03 = 'product_group_name'
  s_04 = 'garment_group_name'

  s_1 = 'colour' # keep element if colour is in
  s_2 = 'name' # not to keep element if name is in AND colour not in


  # delete columns with 'name' in it, except if 'colour' is in it
  rem_cols_1 = [element for element in cols if
   (s_2 not in element) or
    (s_2 and s_1 in element) or
     (s_01 in element) or
      (s_02 in element) or
       (s_03 in element) or
        (s_04 in element)]

  # delete columns with 'id' in it except if 'id' is associated with 'customer' or 'article'
  s_3 = 'id'
  s_4 = 'customer'
  s_5 = 'article'
  rem_cols_2 = [element for element in rem_cols_1 if (s_3 not in element) or (s_3 and s_4 in element) or (s_3 and s_5 in element)]

  print('nb cols after: {}'.format(len(rem_cols_2)))
  print('cols after: {}'.format(rem_cols_2))

  df_cleaned = df.loc[:,rem_cols_2]

  return df_cleaned




In [ ]:
df_cleaned = supp_other_non_inf(df_glob_after)

### df_cleaned info()

In [ ]:
df_cleaned.info()

- _<font color="red"> --> We see "age" feature has missing quite a lot of missings.</font>_
- _<font color="red"> --> We have to find a way to predict missings.</font>_
- _<font color="red"> --> This is the next step.</font>_



## Question of encoding categorical features; Checking number of unique values.

- This section provide us with information on how to encode cataegorical features.
- If unique values for each feature is about of some elements (not thousands !), then we could go with OneHotEncoder() methodology.
- If not, we will go with text encoding (NLP) methodology.

In [ ]:
def check_unique_val(df, feature):
  return print(f'number of unique values: {df[feature].nunique()}, unique values: {df[feature].unique()}')

In [ ]:
check_unique_val(df_cleaned, 'prod_name')

In [ ]:
check_unique_val(df_cleaned, 'product_type_name')

In [ ]:
check_unique_val(df_cleaned, 'product_group_name')

In [ ]:
check_unique_val(df_cleaned, 'colour_group_name')

In [ ]:
check_unique_val(df_cleaned, 'perceived_colour_value_name')

In [ ]:
check_unique_val(df_cleaned, 'perceived_colour_master_name')

In [ ]:
check_unique_val(df_cleaned,'garment_group_name')

In [ ]:
check_unique_val(df_cleaned,'detail_desc')

- _<font color="red"> --> We have height categorical features.</font>_
- _<font color="red"> --> In order to be consistent in next steps we have to encode these features in order to convert categorical features into numeric ones.</font>_
- _<font color="red"> --> We have to use same methodology for encoding.</font>_
- _<font color="red"> --> We will use embedding methodology using model from Huggingface.</font>_



## **Missings in "age" feature. --> # First try**

- We try a simple impting method using SimpleImputer from sklearn.
- Have to import corresponding modules.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
feature_age = df_cleaned[['age']]
print(f'feature_age.shape: {feature_age.shape}')

X = imp.fit_transform(feature_age)
print(f'X.shape: {X.shape}')
print(f'X null values: {np.isnan(X).sum()}')

df_cleaned['age_imputed'] = X
df_cleaned['age_imputed'].unique()


In [ ]:
fig = px.histogram(df_cleaned['age_imputed'])
fig.show()

- <font color='red'> Conclusion: model not capable to output proper predictions.</font>
- <font color='red'> We see data is not enough distributed after imputation, whatever we use MEAN or MEDIAN</font>

## **Missings in "age" feature. --> # Second try**

- we will try KNN imputer from sklearn.
- Have to import related modules.
- Have to scale features 'age' and 'price'

In [ ]:
df_cleaned.iloc[0:2, -6:]

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
scaler = StandardScaler()

In [ ]:
X_1 = df_cleaned[['age', 'rfm_score']]
X_1_sc = scaler.fit_transform(X_1)
print(f'X_1_sc.shape: {X_1_sc.shape}')
X_1_sc[0:5, :]

In [ ]:
X_price_sc = df_cleaned[['price']].values

X_sc = np.concatenate([X_1_sc, X_price_sc], axis=1)
X_sc[0:5, :]

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights='uniform')

In [ ]:
imputed = imputer.fit_transform(X_sc)

In [ ]:
imputed_inverse = scaler.inverse_transform(imputed[:, 0:2])

In [ ]:
imputed_inverse[0:5, :]

In [ ]:
df_cleaned['age_imputed'] = imputed_inverse[:, 0]

In [ ]:
fig = px.histogram(df_cleaned['age_imputed'])
fig.show()

In [ ]:
df_cleaned['age_imputed'].isnull().mean()

In [ ]:
df_cleaned_1 = df_cleaned.drop(['age'], axis=1)
df_cleaned_1.rename(columns={'age_imputed': 'age'}, inplace=True)
df_cleaned_1.isnull().mean()

- <font color='red'> Conclusion: we keep that methodology for age feature prediction as data distribution is more in line with expectations.</font>


## **Missings in "detail_desc" feature.**

- We know there are missing values in "detail_desc".
- These missing values ​​correspond to a value of the article_id value and are intended to give a qualitative description of the article.
- If we can find that same article_id somewhere in the dataset, then we can get the missing article_id.
- In the other way, if we cannot find that "article_id" elsewhere in the dataset, then it is impossible to deduct "detail_desc" value from other data in the dataset.
- In that case we will delete related rows.

### Divide data set into 'with null values' and 'not null values'.

In [ ]:
df_cleaned_2 = df_cleaned_1[df_cleaned_1['detail_desc'].notna()]
df_cleaned_2.info()
print()
"""
Once embedding process is done, we will need to gather
df_cleaned_2 which is not null, with the rest of the data.
"""
df_cleaned_2_withna = df_cleaned_1[df_cleaned_1['detail_desc'].isna()]
df_cleaned_2_withna.info()

In [ ]:
"""
build list of article_id with missing values.
then see if there are any article_id in df_cleaned_2
"""
print('---------')
list_article_id_withna = list(df_cleaned_2_withna['article_id'])
df_try = df_cleaned_2['article_id'].isin(list_article_id_withna)
print(f'check if we can find "detail_desc" missing values somewhere in dataset: {df_try.isnull().sum()}')
print()
if df_try.isnull().sum() == 0:
  print(f'we cannot find "detail_desc" missing values somewhere in dataset,\n we have to delete rows with missing values.\n we are left with df_cleaned_2 only')
print('---------')

### df_cleaned_2.info().

In [ ]:
"""Therefore, after deletion we are left with "df_cleaned_2"""
df_cleaned_2.info()

In [ ]:
cleaned_cols = list(df_cleaned_2.columns)
print(f'cleaned_cols -- names: {cleaned_cols}, \n number -- {len(cleaned_cols)}')

# **Embeddings.**

## Importing modules.

In [ ]:
#!pip install sentence_transformers
!pip install -U sentence-transformers

## Model for embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer
"https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2"
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## Select categorical features & non categorical features.

In [ ]:
cat_cols = list(df_cleaned_2.select_dtypes(include=['object']))
print(f'cat_cols: {cat_cols}, \n nb: {len(cat_cols)}')

non_cat_cols = list(df_cleaned_2.select_dtypes(exclude=['object']))
print(f'non_cat_cols: {non_cat_cols}, \n nb: {len(non_cat_cols)}')

## Down sampling for memory limitation constraint.

- We will only consider customer who are active.
- Active means: at least one purchase per month.
- If non purchase during the month, customer will be taken out of dataset.


In [ ]:
"""Filter on rows for which customer's purchases are less than 10 ITEMS PER MONTH"""

def filter_on_purchase(df):
  print(f'nb of rows before deletion: {len(df)}')

  df_final = pd.DataFrame()
  d = {}

  """build monthly periods"""
  start_date = df['t_dat'].min()
  end_date = df['t_dat'].max()
  nb_of_months = (end_date - start_date).days//30
  print(f'nb_of_months: {nb_of_months}')

  for month in range(nb_of_months):
    print(f'{month}')
    if start_date + pd.Timedelta(days=30 * month) <= df['t_dat'].max():

      end_date = start_date + pd.Timedelta(days=30 * month) + pd.Timedelta(days=30)
      #print(f'start_date: {start_date + pd.Timedelta(days=30 * month)}')
      #print(f'end_date: {end_date}')

      df_temp_1 = df[(df['t_dat'] >= start_date + pd.Timedelta(days=30 * month)) & (df['t_dat'] < end_date)]
      print(f'df_temp_1.shape: {df_temp_1.shape}')

      """list of indexes for which customer's purchases are less than 10 per month"""
      list_of_indexes = list(df_temp_1.groupby(['customer_id'])['article_id'].filter(lambda x: x.count() <= 10).index)
      print(f'list_of_indexes: {len(list_of_indexes)}')


      """delete rows"""
      df_filtered = df_temp_1.drop(list_of_indexes)
      print(f'df_filtered.shape: {df_filtered.shape}')


      """append df_filtered to df_final"""
      df_final = pd.concat([df_final, df_filtered])
      print(f'df_final.shape: {df_final.shape}')
      df_final = df_final.reset_index(drop=True)

      """fill d"""
      d[month] = len(list_of_indexes)

  df_del_rows = pd.DataFrame.from_dict(d, orient='index')
  df_del_rows.rename(columns={0: 'deleted_rows_per_month'}, inplace=True)
  df_del_rows.index.name = 'month'



  return df_del_rows, df_final


In [ ]:
df_del_rows, df_cleaned_2_masked = filter_on_purchase(df_cleaned_2)

In [ ]:
sns.lineplot(data=df_del_rows, x=df_del_rows.index, y='deleted_rows_per_month')
plt.title('nb of deleted rows per Month')
plt.xlabel('period')
plt.ylabel('nb of deleted rows')
plt.show()

## df_cleaned_2masked info()

In [ ]:
df_cleaned_2_masked.info()

In [ ]:
def check_buyings(df):
  """this function is used to take customers out of the dataset if they do not
  purchase at least 10 ITEMS PER WEEK.
  """

  print(f'nb rows before deletion: {len(df)}')
  len_rows_to_be_deleted = 0
  days_increment= 7
  df_final = pd.DataFrame()
  d = {}

  start_date = df['t_dat'].min()
  max_date = df['t_dat'].max()
  nb_of_days = (max_date - start_date).days
  nb_of_periods_of_7_days = nb_of_days//days_increment

  """prints"""
  print(f'nb_of_days: {nb_of_days}')
  print(f'nb_of_periods_of_7_days: {nb_of_periods_of_7_days}')


  for period in range(0, nb_of_periods_of_7_days):

    start_date_period = start_date
    end_date = start_date_period + pd.Timedelta(days=days_increment)
    df_temp_1 = df[(df['t_dat'] >= start_date_period) & (df['t_dat'] < end_date)]

    """prints"""
    print('---')
    print(f'start_date_period: {start_date_period}')
    print(f'period: {period}, nb days -- {pd.Timedelta(days=days_increment * period)}')
    print(f'end_date: {end_date}')
    print()
    print(f'{df_temp_1["t_dat"].min()}')
    print(f'{df_temp_1["t_dat"].max()}')
    #print((f'df_temp_1: {df_temp_1[0:4]}'))

    """list of indexes for which customer's purchases are less than 1 per month"""
    list_of_indexes = list(df_temp_1.groupby(['customer_id'])['article_id'].filter(lambda x: x.count() <= 10).index)
    print(f'list_of_indexes: {len(list_of_indexes)}')

    """append nb of deleted rows to d
    """
    d[period] = len(list_of_indexes)


    """delete rows"""
    df_filtered = df_temp_1.drop(list_of_indexes)
    print(f'df_filtered.shape: {df_filtered.shape}')


    """append df_filtered to df_final"""
    df_final = pd.concat([df_final, df_filtered])
    print(f'df_final.shape: {df_final.shape}')
    df_final = df_final.reset_index(drop=True)

    start_date = end_date

  """put d into df"""
  print(f'd: {d}')
  df_del_rows = pd.DataFrame.from_dict(d, orient='index')
  df_del_rows.rename(columns={0: 'deleted_rows_per_period'}, inplace=True)
  df_del_rows.index.name = 'period'
  print(f'df_del_rows[0:5]: {df_del_rows[0:5]}')
  #print(f'df_del_rows.shape: {df_del_rows.shape}')

  print(f'nb rows after deletion: {len(df_final)}')

  return df_del_rows, df_final


In [ ]:
df_del_rows, df_cleaned_2_reduced = check_buyings(df_cleaned_2_masked)

In [ ]:
sns.lineplot(data=df_del_rows, x=df_del_rows.index, y='deleted_rows_per_period')
plt.title('nb of deleted rows per period of seven days')
plt.xlabel('period')
plt.ylabel('nb of deleted rows')
plt.show()

In [ ]:
df_cleaned_2_reduced.info()

## Drop some features.

- Features not to be used in next setps.

In [ ]:
"""drop some features we do not need"""
features_to_be_dropped = ['perceived_colour_value_name','perceived_colour_master_name',
                          'recency_score', 'frequency_score','monetary_score',
                         'rfm_score', 'clusters', 'age_cat', 'cum_sales',
                         'sales_by_customer']
df_cleaned_2_reduced = df_cleaned_2_reduced.drop(features_to_be_dropped, axis=1)

## df_cleaned_2_reduced.info()

In [ ]:
df_cleaned_2_reduced.info()

## Graphs: Best customer behavior.

Best customers analysis:
- how many customers ?
- In average, customer spending ?
- In average, number of articles purchased per week per customer ?
- Which "product_type_name" is purchased most ?
- Which "product_group_name" is purchased most ?   
- Wich "colour_group_name" is purchased most ?
- Wich "garment_group_name" is purchased most ?
- ...


In [ ]:
nb_customers = df_cleaned_2_reduced['customer_id'].nunique()
print(f'nb_customers: {nb_customers}')

In [ ]:
spendings = df_cleaned_2_reduced.groupby('customer_id')['price'].sum()
print(f'spendings.shape: {spendings.shape}')
fig = px.histogram(spendings, title='spending by customer')
fig.show()

top_100_spendings = spendings.sort_values(ascending=False)[0:100]
print(f'top_100_spendings.shape: {top_100_spendings.shape}')
fig = px.histogram(top_100_spendings, title='spending for top 100 customers')
fig.show()

In [ ]:
purchasings = df_cleaned_2_reduced.groupby('customer_id')['article_id'].count()
print(f'purchasings.shape: {purchasings.shape}')
fig = px.histogram(purchasings, title='nb of articles purchased by customer')
fig.show()

top_100_purchasings = purchasings.sort_values(ascending=False)[0:100]
print(f'top_100_purchasings.shape: {top_100_purchasings.shape}')
fig = px.histogram(top_100_purchasings, title='nb of articles purchased by top 100 customers')
fig.show()

In [ ]:
def display_prod_types(feature):
  prod_type_name = df_cleaned_2_reduced[feature]
  fig = px.histogram(prod_type_name, title=feature)
  fig.show()

  most_purchased_prod_type = prod_type_name.value_counts().index[0:50]
  #print(f'most_purchased_prod_type: {most_purchased_prod_type}')
  fig = px.bar(prod_type_name.value_counts(), title='most purchased' + ' ' + feature)
  fig.show()
  return

In [ ]:
display_prod_types('product_type_name')

In [ ]:
display_prod_types('product_group_name')

In [ ]:
display_prod_types('colour_group_name')

In [ ]:
display_prod_types('garment_group_name')

In [ ]:
fig = px.bar(df_glob['index_name'].value_counts(), title='nb purchases by index name', color=df_glob['index_name'].unique())

fig.show()

In [ ]:
fig = px.bar(df_glob['department_name'].value_counts(), title='nb purchases by dpt. name')

fig.show()

In [ ]:
counts_per_customer = df_cleaned_2_reduced.groupby(['customer_id'],
                                                   as_index=False)['article_id'].count().sort_values(by=['article_id'], ascending=False)[0:100]

counts_per_customer.rename(columns={'article_id': 'nb of articles'}, inplace=True)
counts_per_customer.set_index('customer_id', inplace=True)

In [ ]:
mean_counts_per_cust = counts_per_customer.mean()

In [ ]:
ax = sns.barplot(x=counts_per_customer.index,
            y=counts_per_customer['nb of articles'],
                 color='green')
ax.set(xlabel='customer_id', ylabel='nb of articles')
ax.set_title('nb of articles purchased by top 100 customers')
#ax.tick_params(labelrotation=45)
ax.axhline(y = mean_counts_per_cust[0],
           color = 'r',
           linestyle = 'dashed',
           xmin = 0.1,
           xmax = 0.9)
plt.title('nb of articles purchased by top 100 customers vs mean')
plt.xlabel('customer_id')
plt.ylabel('nb of articles')

plt.show()

## Get embeddings.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def tolist_fn(df, feature):
  """get unique values of a feature"""
  unique_val = df[feature].unique()
  print(f'unique_val.shape: {len(unique_val)}')
  feature_as_list = unique_val.tolist()
  print(f'feature_as_list.shape: {len(feature_as_list)}')
  return feature_as_list


In [ ]:
detail_desc_tolist = tolist_fn(df_cleaned_2_reduced, 'detail_desc')

In [ ]:
garment_type_tolist = tolist_fn(df_cleaned_2_reduced, 'garment_group_name')

In [ ]:
color_group_name_tolist = tolist_fn(df_cleaned_2_reduced, 'colour_group_name')

In [ ]:
detail_desc_vect = model.encode(detail_desc_tolist)
print(f'detail_desc_vect.shape: {detail_desc_vect.shape}')

In [ ]:
garment_vect = model.encode(garment_type_tolist)
print(f'garment_vect.shape: {garment_vect.shape}')

In [ ]:
color_vect = model.encode(color_group_name_tolist)
print(f'color_vect.shape: {color_vect.shape}')

## Dimension reduction.

In [ ]:
def dim_red(vect):
  n_components = vect.shape[1]//32
  pca = PCA(n_components=n_components)
  vect_pca = pca.fit_transform(vect)
  print(f'vect_pca.shape: {vect_pca.shape}')

  return n_components,vect_pca

In [ ]:
n_components, pca_detail_desc = dim_red(detail_desc_vect)
print(f'pca_detail_desc.shape: {pca_detail_desc.shape}')

In [ ]:
n_components, pca_garment = dim_red(garment_vect)
print(f'pca_garment.shape: {pca_garment.shape}')

In [ ]:
n_components, pca_color = dim_red(color_vect)
print(f'pca_color.shape: {pca_color.shape}')

In [ ]:
df_cleaned_2_reduced.shape

In [ ]:
df_cleaned_2_reduced.info()

In [ ]:
df_cleaned_2_reduced.values[0:2, 9:10]

In [ ]:
def last_two_weeks(df):

  """ As per product recommendation request in the next seven days following the most recent date in dataset,
  we will reduce dataset period to the last two weeks"""

  """doing so, we have most recent article purchases made by customers"""

  start_date = df['t_dat'].max() - pd.Timedelta(days=2 * 7)
  end_date = df['t_dat'].max()
  new_df = df[(df['t_dat'] >= start_date) & (df['t_dat'] <= end_date)]
  print(f'new_df.shape: {new_df.shape}')

  """after reducing time period, we are going to
  drop duplicates if article_id is the same"""
  new_df = new_df.drop_duplicates(subset=['article_id'])
  print(f'new_df.shape: {new_df.shape}')
  print(f'new_df[0:2] -- {new_df[0:2]}')

  return new_df


In [ ]:
df_3 = last_two_weeks(df_cleaned_2_reduced)
df_3.shape

In [ ]:
"""
from previous step, we see pca results have different dimensions
to be able to concatenate those results, we have to get these same dimensions.
this is the objective with this function.
"""

def retrieve_item(df):

  """l is a list of unique text items, seen in previous step, been embedded"""
  features = ['detail_desc', 'garment_group_name', 'colour_group_name']

  """pca_mat results from text list embedding"""
  pca_mat = [pca_detail_desc, pca_garment, pca_color]

  """list_of_arrays is used to append outputed arrays"""
  list_of_arrays = []

  for feature in features:
    print(f'feature {feature}')

    """get mapping between embeddings and text items in dataframe"""
    l = df[feature].unique().tolist()

    idx_in_features = features.index(feature)

    arr = df[[feature]].values

    """create array with same dims than arr, will store embeddings"""
    new_arr = np.zeros((arr.shape[0], n_components))

    for item in l:
      idx_in_l = l.index(item)

      """get index of item in arr
      np.where() function outputs two indices arrays:
      one for rows + one for column"""

      idx = np.where(arr == item)[0] # take only related rows indices array!
      new_arr[idx] = pca_mat[idx_in_features][idx_in_l, :]

    list_of_arrays.append(new_arr)
  final_array = np.stack(list_of_arrays, axis=1)
  print(f'final_array.shape: {final_array.shape}')

  return final_array


In [ ]:
final_array = retrieve_item(df_3)

In [ ]:
final_array[0:2, :, :]

# Cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

def reduce_dim(df, mat, nb_batches=100):
  """create article_id_temp array to be able to stack article_id with the array
  that have different dimensions """

  article_id_temp = np.zeros((df.shape[0], 36))
  article_id = df[['article_id']].values
  article_id_temp[:, 0] = article_id[:, 0]

  """flatten the array: three features will be concatenated to produce one single pattern"""
  new_mat = np.reshape(mat, (mat.shape[0], 3 * 12))
  print(f'shape of mat after feature concatenating -- {new_mat.shape}')

  """stack article_id and new_mat"""
  new_mat = np.stack([article_id_temp, new_mat], axis=1)
  print(f'shape of mat after stacking article_id_temp -- {new_mat.shape}')
  print()

  cos_sim = cosine_similarity(new_mat[:, 1, :])
  print(f'cos_sim.shape: {cos_sim.shape}')
  print(f'cos_sim[0:2, :] -- {cos_sim[0:2, :]}')


  return cos_sim, new_mat


In [ ]:
cos_sim, new_mat = reduce_dim(df_3, final_array)
print(f'cos_sim.shape: {cos_sim.shape}, new_mat.shape: {new_mat.shape}')

## Function to retrieve best cosine similarity scores.

In [ ]:
"""some article_id values"""
some_art_id = list(df_3['article_id'][0:5])
some_art_id

In [ ]:
def get_best_cos_sim(article_id, cos_sim, new_mat):
  idx_article_id = np.where(new_mat[:, 0, :] == article_id)[0]
  best_idx = np.argsort(cos_sim[idx_article_id])[0][-6:-1]
  print(f'indices of ascending values -- {best_idx}')
  print()

  #print(f'article_id that correspond to indices -- {new_mat[best_idx, 0, :]}')
  list_of_most_similar = [item[0].astype('int64') for item in new_mat[best_idx, 0, :]]
  print(f'list_of_most_similar -- {list_of_most_similar}')
  print('---')

  """display most similar articles"""
  most_similar_articles = df_3[df_3['article_id'].isin(list_of_most_similar)]
  d = {}
  d[article_id] = list_of_most_similar
  return most_similar_articles, d


In [ ]:
df_most_similar, d_most_similar = get_best_cos_sim(some_art_id[1], cos_sim, new_mat)

In [ ]:
print(f'article_id input & most similar products: {d_most_similar}')

In [ ]:
df_most_similar